#  Nsynth autoencoder for fine-tuning

This is draft notebook to test NSynth autoencoder model

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
import os
from tensorflow.python import pywrap_tensorflow
slim = tf.contrib.slim

from model_units import *
import librispeech_reader 


/home/julia/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/julia/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [2]:
class Config(object):
    def __init__(self, train_path = None):
        self.num_iters = 200000
        self.learninig_rate = 1e-3
        self.ae_hop_length = 512
        self.ae_bottleneck_width = 16
        self.train_path = train_path
        
    def get_batch(self, batch_size):
        print('Get batch')
        assert self.train_path is not None
        data_train = librispeech_reader.LibriSpeechDataset(
            self.train_path, is_training = True)
        return data_train.get_wavenet_batch(batch_size, length = 39936)
    
    @staticmethod
    def _condition(x, encoding):
        """Condition the input on the encoding.
        Args:
          x: The [mb, length, channels] float tensor input.
          encoding: The [mb, encoding_length, channels] float tensor encoding.
        Returns:
          The output after broadcasting the encoding to x's shape and adding them.
        """
        mb, length, channels = x.get_shape().as_list()
        enc_mb, enc_length, enc_channels = encoding.get_shape().as_list()
        assert enc_mb == mb
        assert enc_channels == channels

        encoding = tf.reshape(encoding, [mb, enc_length, 1, channels])
        x = tf.reshape(x, [mb, enc_length, -1, channels])
        x += encoding
        x = tf.reshape(x, [mb, length, channels])
        x.set_shape([mb, length, channels])
        return x
    
    
    
    
    def build(self, inputs, is_training):
        ''' 
        Args:
          inputs: A dict of inputs. For training, should contain 'wav'.
        '''
        del is_training
        num_stages = 10
        num_layers = 30
        filter_length = 3
        width = 512
        ae_num_stages = 10
        ae_num_layers = 30
        ae_filter_length = 3
        ae_width = 128
        skip_width = 256
        
        # Encode the source with 8-bit Mu-Law.
        x = inputs['wav']
        x_quantized = mu_law(x)
        with tf.name_scope("scale_and_expand_dims"):
            x_scaled = tf.cast(x_quantized, tf.float32) / 128.0
            x_scaled = tf.expand_dims(x_scaled, 2)
            
        
        ###
        # The Non-Causal Temporal Encoder.
        ###
        en = conv1d(x_scaled,
                    causal=False,
                    num_filters=ae_width,
                    filter_length=ae_filter_length, dilation = 1,
                    name='ae_startconv')

        for num_layer in range(ae_num_layers):
            with tf.name_scope('layer_{}'.format(num_layer)):
                dilation = 2**(num_layer % ae_num_stages)

                d = tf.nn.relu(en)
                d = conv1d(d,
                           causal=False,
                           num_filters=ae_width,
                           filter_length=ae_filter_length,
                           dilation=dilation,
                           name='ae_dilatedconv_%d' % (num_layer + 1))
                d = tf.nn.relu(d)
                en += conv1d(d,
                             num_filters=ae_width,
                             filter_length=1,
                             name='ae_res_%d' % (num_layer + 1))

        en = conv1d(en,
                    num_filters=self.ae_bottleneck_width,
                    filter_length=1,
                    name='ae_bottleneck')
        en = pool1d(en, self.ae_hop_length, name = 'ae_pool', mode = 'avg')
        encoding = en
                    
        
        ###
        # Classification layers.
        ###
        en_shape = encoding.get_shape().as_list()
        clf_hu_size1 = 256
        clf_hu_size2 = 128
        clf_output_size = 40
        
        clf = tf.reshape(en, [-1, en_shape[-1]*en_shape[-2]])
        clf = fc_layer(clf,  en_shape[-1]*en_shape[-2], clf_hu_size1, "clf_fc1")
        clf = tf.nn.relu(clf, name = "clf_relu1")
        clf = tf.nn.dropout(clf, keep_prob = 0.5)
        clf = fc_layer(clf,  clf_hu_size1, clf_hu_size2, "clf_fc2")
        clf = tf.nn.relu(clf, name = "clf_relu2")
        
        
             ###
        # The WaveNet Decoder.
        ###
        l = shift_right(x_scaled)
        l = conv1d(l, num_filters=width, filter_length=filter_length, name='startconv')

        # Set up skip connections.
        s = conv1d(l, num_filters=skip_width, filter_length=1, name='skip_start')

        # Residual blocks with skip connections.
        for i in range(num_layers):
            dilation = 2**(i % num_stages)
            d = conv1d(
              l,
              num_filters=2 * width,
              filter_length=filter_length,
              dilation=dilation,
              name='dilatedconv_%d' % (i + 1))
            d = self._condition(d,conv1d(
                                  en,
                                  num_filters=2 * width,
                                  filter_length=1,
                                  name='cond_map_%d' % (i + 1)))

            assert d.get_shape().as_list()[2] % 2 == 0
            m = d.get_shape().as_list()[2] // 2
            d_sigmoid = tf.sigmoid(d[:, :, :m])
            d_tanh = tf.tanh(d[:, :, m:])
            d = d_sigmoid * d_tanh

            l += conv1d(
              d, num_filters=width, filter_length=1, name='res_%d' % (i + 1))
            s += conv1d(
              d, num_filters=skip_width, filter_length=1, name='skip_%d' % (i + 1))

        s = tf.nn.relu(s)
        s = conv1d(s, num_filters=skip_width, filter_length=1, name='out1')
        s = self._condition(s, conv1d(
                                en,
                                num_filters=skip_width,
                                filter_length=1,
                                name='cond_map_out1'))
        s = tf.nn.relu(s)

        ###
        # Compute the logits and get the loss for classifier.
        ###
        logits1 = fc_layer(clf,  clf_hu_size2, clf_output_size, "clf_logits1")
        probs1 = tf.nn.softmax(logits1, name = 'clf_softmax1')

#         logits2 = fc_layer(clf,  clf_hu_size2, 2, "clf_fc2")
#         probs2 = tf.nn.softmax(logits2, name = 'clf_softmax2')

     
        with tf.name_scope('clf_loss'):
            loss1 = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=logits1, labels=inputs['label'], name='nll'), 0, name='clf_loss1')

#             loss2 = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
#                 logits=logits2, labels=inputs['sex'], name='nll'), 0, name='loss2')

#             loss = loss1
#             loss = loss1 + loss2

        
        ###
        # Compute the logits and get the loss for decoder. 
        ###
        logits_dec = conv1d(s, num_filters=256, filter_length=1, name='dec_logits')
        logits_dec = tf.reshape(logits_dec, [-1, 256])
        probs_dec = tf.nn.softmax(logits_dec, name='dec_softmax')
        x_indices = tf.cast(tf.reshape(x_quantized, [-1]), tf.int32) + 128
        loss_dec = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=logits_dec, labels=x_indices, name='nll'),
            0,
            name='dec_loss')
        
        loss = loss1 + loss_dec

        return {'predictions_speaker' : logits1,
#                 'predictions_sex' : logits2,
                'loss': loss,
                'eval': {'nll':loss}, 
                'quantized_input' : x_quantized,
                'encoding':encoding,
               }

In [3]:
LOGDIR = "/home/julia/DeepVoice_project/LOGDIR/"
OLD_MODEL_PATH = '/home/julia/DeepVoice_data/wavenet-ckpt/model.ckpt-200000'
# OLD_MODEL_PATH = '/home/julia/DeepVoice_data/wavenet-voice-ckpt/model.ckpt-200000'

# "Batch size spread across all sync replicas.We use a size of 32."
TOTAL_BATCH_SIZE = 1

# "Number of replicas. We train with 32."
WORKER_REPLICAS = 1

# "Number of tasks in the ps job. If 0 no ps job is used. We typically use 11 "
PS_TASKS = 0

wav_path = '/home/julia/DeepVoice_project/LibriSpeech_small'
tfrecord_path = '{}/wavs.tfrecord'.format(wav_path)

# "The path to the train tfrecord"
TRAIN_PATH = tfrecord_path 

#  "Task id of the replica running the training."
TASK = 0

In [ ]:
model = Config(train_path = TRAIN_PATH)   

with tf.Graph().as_default():
    logdir = LOGDIR
    total_batch_size = TOTAL_BATCH_SIZE
    worker_replicas = WORKER_REPLICAS
    assert total_batch_size % worker_replicas == 0
    worker_batch_size = int(total_batch_size / worker_replicas)
    
    
    # Run the Reader on the CPU
    cpu_device = "/job:localhost/replica:0/task:0/CPU:0"
    if PS_TASKS:
        cpu_device = "/job:worker/CPU:0"
        
    # Get input batch for worker
    with tf.device(cpu_device):
        inputs_dict = model.get_batch(worker_batch_size)
        
    
    with tf.device( tf.train.replica_device_setter(
        ps_tasks=PS_TASKS, merge_devices=True)):
        
        global_step = tf.get_variable(
            "global_step", [],
            tf.int32,
            initializer=tf.constant_initializer(0),
            trainable=False)
        
        lr = model.learninig_rate
        tf.summary.scalar("learning_rate", lr)
        
        print('Lets get output')
        outputs_dict = model.build(inputs_dict, is_training = True)
        loss = outputs_dict["loss"]
        tf.summary.scalar("train_loss", loss)
        
#         # if we don't want to use moving average while averaging gradients from replicas 
#         opt = tf.train.SyncReplicasOptimizer(
#             tf.train.AdamOptimizer(lr, epsilon = 1e-8),
#             replicas_to_aggregate = worker_replicas,
#             total_num_replicas = worker_replicas)

        # if we want to use moving average while averaging gradients from replicas 
        ema = tf.train.ExponentialMovingAverage(
          decay=0.9999, num_updates=global_step)
        opt = tf.train.SyncReplicasOptimizer(
            tf.train.AdamOptimizer(lr, epsilon=1e-8),
            replicas_to_aggregate = worker_replicas,
            total_num_replicas=worker_replicas,
            variable_averages=ema,
            variables_to_average=tf.trainable_variables())

        train_op = opt.minimize(
            loss,
            global_step = global_step,
            name = "train",
            colocate_gradients_with_ops = True)
    
        session_config = tf.ConfigProto(allow_soft_placement = True)
        
        ###
        # Initialize ae layers with corresponding weights from pre-trained wavenet autoencoder
        trainable_var_names = [v.op.name for v in tf.trainable_variables()]
        
        var_names_to_values = {}
        reader = pywrap_tensorflow.NewCheckpointReader(OLD_MODEL_PATH) 
        
        for key in np.intersect1d(trainable_var_names, list(reader.get_variable_to_shape_map().keys())):
            var_names_to_values[key] = reader.get_tensor(key)

        init_assign_op, init_feed_dict = slim.assign_from_values(var_names_to_values)
        
        def InitAssignFn(sess):
            sess.run(init_assign_op, init_feed_dict)
        ###
        
        is_chief = (TASK == 0)
        local_init_op = opt.chief_init_op if is_chief else opt.local_step_init_op

        slim.learning.train(
            train_op=train_op,
            is_chief=is_chief,
            logdir = logdir,
            number_of_steps=model.num_iters,
            global_step=global_step,
            log_every_n_steps=250,
            local_init_op=local_init_op,
            save_interval_secs=300,
            sync_optimizer=opt,
            session_config=session_config,
            init_fn = InitAssignFn)
        

Get batch
Lets get output
INFO:tensorflow:SyncReplicasV2: replicas_to_aggregate=1; total_num_replicas=1
INFO:tensorflow:Starting Session.
INFO:tensorflow:Saving checkpoint to path /home/julia/DeepVoice_project/LOGDIR/model.ckpt
INFO:tensorflow:Starting Queues.
INFO:tensorflow:global_step/sec: 0


In [ ]:
! tensorboard --logdir=/home/julia/DeepVoice_project/LOGDIR/

/home/julia/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/julia/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
TensorBoard 0.4.0rc3 at http://julia-ThinkPad-T470:6006 (Press CTRL+C to quit)


# Notes

In [28]:
reader = pywrap_tensorflow.NewCheckpointReader(OLD_MODEL_PATH)
rdr1= pywrap_tensorflow.NewCheckpointReader(LOGDIR+'model.ckpt-0')
rdr2= pywrap_tensorflow.NewCheckpointReader(LOGDIR+'model.ckpt-10')

reader_keys = reader.get_variable_to_shape_map().keys()
rdr_keys = rdr1.get_variable_to_shape_map().keys()
print(len(reader_keys), len(rdr_keys))

l = [key for key in reader_keys if key.startswith('ae')]
ll = [key for key in rdr_keys if key.startswith('ae')]
print(len(l), len(ll))

# Lets show that we loaded weights from the old model correctly
print(reader.get_tensor('ae_bottleneck/W') == rdr1.get_tensor('ae_bottleneck/W'))

# Lets show that the weights change while training
for key in ['fc1/W', 'ae_bottleneck/W']:
    print(rdr1.get_tensor('global_step'), rdr2.get_tensor('global_step'))
    print(rdr1.get_tensor(key) == rdr2.get_tensor(key))

1495 511
496 496
[[[[ True  True  True ...  True  True  True]
   [ True  True  True ...  True  True  True]
   [ True  True  True ...  True  True  True]
   ...
   [ True  True  True ...  True  True  True]
   [ True  True  True ...  True  True  True]
   [ True  True  True ...  True  True  True]]]]
0 10
[[False False False  True]
 [False False False False]
 [False False False  True]
 ...
 [False False False False]
 [False False False False]
 [False False False  True]]
0 10
[[[[False False False ... False False False]
   [False False False ... False False False]
   [False False False ... False False False]
   ...
   [False False False ... False False False]
   [False False False ... False False False]
   [False False False ... False False False]]]]


In [29]:
np.setxor1d(list(rdr_keys), ll)

array(['beta1_power', 'beta2_power', 'fc1/B', 'fc1/B/Adam',
       'fc1/B/Adam_1', 'fc1/B/ExponentialMovingAverage', 'fc1/W',
       'fc1/W/Adam', 'fc1/W/Adam_1', 'fc1/W/ExponentialMovingAverage',
       'fc2/B', 'fc2/B/ExponentialMovingAverage', 'fc2/W',
       'fc2/W/ExponentialMovingAverage', 'global_step'], dtype='<U49')

In [30]:
print(inputs_dict)
print(outputs_dict)

{'wav': <tf.Tensor 'Reshape_1:0' shape=(1, 39936) dtype=float32>, 'sr': <tf.Tensor 'strided_slice:0' shape=(1,) dtype=int32>, 'speaker': <tf.Tensor 'strided_slice_1:0' shape=(1,) dtype=int32>}
{'predictions_speaker': <tf.Tensor 'fc1/add:0' shape=(1, 4) dtype=float32>, 'predictions_sex': <tf.Tensor 'fc2/add:0' shape=(1, 2) dtype=float32>, 'loss': <tf.Tensor 'loss/loss1:0' shape=() dtype=float32>, 'eval': {'nll': <tf.Tensor 'loss/loss1:0' shape=() dtype=float32>}, 'quantized_input': <tf.Tensor 'mu_law/Floor:0' shape=(1, 39936) dtype=float32>, 'encoding': <tf.Tensor 'ae_pool/Reshape_1:0' shape=(1, 78, 16) dtype=float32>}


In [9]:
def ae_model(learning_rate, hparam):

  
    x = tf.placeholder(tf.float32, shape=[None,  1024*10], name="x")
    labels1 = tf.placeholder(tf.int32, shape = [None], name = 'labels1')
    labels2 = tf.placeholder(tf.int32, shape = [None], name = 'labels2')
    
    print('x initial shape: {}'.format(x.get_shape().as_list()))

    x_quantized = mu_law(x)
    with tf.name_scope("scale_and_expand_dims"):
        x_scaled = tf.cast(x_quantized, tf.float32) / 128.0
        x_scaled = tf.expand_dims(x_scaled, 2)

    print('x converted')
    print('x scaled shape: {}'.format(x_scaled.get_shape().as_list()))

    LOGDIR = "/home/julia/DeepVoice_project/LOGDIR/"
    CHECKPOINT = '/home/julia/magenta-demos/jupyter-notebooks/wavenet-ckpt/model.ckpt-200000'


    ae_filter_length = 3
    ae_width = 128
    ae_bottleneck_width = 16
    ae_hop_length = 512


    en = conv1d(x_scaled,
                causal=False,
                num_filters=ae_width,
                filter_length=ae_filter_length, dilation = 1,
                name='ae_startconv')

    ae_num_layers = 30
    ae_num_stages = 10
    for num_layer in range(ae_num_layers):
        with tf.name_scope('layer_{}'.format(num_layer)):
            dilation = 2**(num_layer % ae_num_stages)

            d = tf.nn.relu(en)
            d = conv1d(d,
                       causal=False,
                       num_filters=ae_width,
                       filter_length=ae_filter_length,
                       dilation=dilation,
                       name='ae_dilatedconv_%d' % (num_layer + 1))
            d = tf.nn.relu(d)
            en += conv1d(d,
                         num_filters=ae_width,
                         filter_length=1,
                         name='ae_res_%d' % (num_layer + 1))

    en = conv1d(en,
                num_filters=ae_bottleneck_width,
                filter_length=1,
                name='ae_bottleneck')
    en = pool1d(en, ae_hop_length, name = 'ae_pool', mode = 'avg')

    en_shape = en.get_shape().as_list()
    flattened = tf.reshape(en, [-1, en_shape[-1]*en_shape[-2]])
    
#     embedding_input = fc_layer(flattened,  en_shape[-1]*en_shape[-2], 6, "fc")
#     embedding_size = 6


    logits1 = fc_layer(flattened,  en_shape[-1]*en_shape[-2], 4, "fc1")
    probs1 = tf.nn.softmax(logits1, name = 'softmax1')

    logits2 = fc_layer(flattened,  en_shape[-1]*en_shape[-2], 2, "fc2")
    probs2 = tf.nn.softmax(logits2, name = 'softmax2')

    with tf.name_scope('loss'):
        loss1 = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits1, labels=labels1, name='nll'), 0, name='loss1')

        loss2 = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits2, labels=labels2, name='nll'), 0, name='loss2')
        
        loss = loss1
        loss = loss1 + loss2
        tf.summary.scalar('loss', loss)

    learning_rate = 1e-3
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)
#     with tf.name_scope("train"):
#         train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)

    summ = tf.summary.merge_all()
    
#     embedding = tf.Variable(tf.zeros([10, embedding_size]), name = 'test_embedding')
#     assignment = embedding.assign(embedding_input)
#     print(embedding_input.shape)
    
    
    MODEL_PATH = '/home/julia/magenta-demos/jupyter-notebooks/wavenet-ckpt/model.ckpt-200000'
#     MODEL_PATH = "/home/julia/DeepVoice_project/LOGDIR/model.ckpt-2"    
    
    all_variables = tf.global_variables()
    
    saver = tf.train.Saver(all_variables[:1])
#     saver = tf.train.Saver()
   
    
    with tf.Session() as sess:
    
        tf.variables_initializer(all_variables[1:]).run()
        saver.restore(sess, MODEL_PATH)

#         sess.run(tf.global_variables_initializer())


        writer = tf.summary.FileWriter(LOGDIR + hparam)
        writer.add_graph(sess.graph)


    #     config = tf.contrib.tensorboard.plugins.projector.ProjectorConfig()
    #     embedding_config = config.embeddings.add()
    #     embedding_config.tensor_name = embedding.name
    #     LABELS = os.path.join(os.getcwd(), "labels_10.tsv")

    #     embedding_config.metadata_path = LABELS
    #     tf.contrib.tensorboard.plugins.projector.visualize_embeddings(writer, config)   

#         for i in range(3):
#             print('\n________________Step {}________________'.format(i))


#             batch = [np.zeros((10, 1024*10)),
#                      np.random.choice(np.arange(4), size = 10),
#                      np.random.choice(np.arange(2), size = 10)]
# #             if i%1 == 0:
#     # #             print('\n________________Step {}________________'.format(i))
#     #             [s] = sess.run([summ], feed_dict = {x: batch[0], labels1: batch[1], labels2: batch[2]})
#     #             writer.add_summary(s, i)
#             if i%2 == 0:
# #                 sess.run(assignment, feed_dict={x: batch[0], labels1: batch[1], labels2: batch[2]})

#                 saver.save(sess, os.path.join(LOGDIR, "model.ckpt"), i)

#             sess.run(train_step, feed_dict = {x: batch[0],  labels1: batch[1], labels2: batch[2]})

In [10]:
hparam = 'ae'

print('Starting run for %s' % hparam)
tf.reset_default_graph()
ae_model(1e-3, hparam)
# print('Done training!')

Starting run for ae
x initial shape: [None, 10240]
x converted
x scaled shape: [None, 10240, 1]

_______ae_startconv_______

_______ae_dilatedconv_1_______

_______ae_res_1_______

_______ae_dilatedconv_2_______

_______ae_res_2_______

_______ae_dilatedconv_3_______

_______ae_res_3_______

_______ae_dilatedconv_4_______

_______ae_res_4_______

_______ae_dilatedconv_5_______

_______ae_res_5_______

_______ae_dilatedconv_6_______

_______ae_res_6_______

_______ae_dilatedconv_7_______

_______ae_res_7_______

_______ae_dilatedconv_8_______

_______ae_res_8_______

_______ae_dilatedconv_9_______

_______ae_res_9_______

_______ae_dilatedconv_10_______

_______ae_res_10_______

_______ae_dilatedconv_11_______

_______ae_res_11_______

_______ae_dilatedconv_12_______

_______ae_res_12_______

_______ae_dilatedconv_13_______

_______ae_res_13_______

_______ae_dilatedconv_14_______

_______ae_res_14_______

_______ae_dilatedconv_15_______

_______ae_res_15_______

_______ae_dilatedconv_1

In [10]:
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file

MODEL_PATH = "/home/julia/DeepVoice_project/LOGDIR/model.ckpt-2"
print_tensors_in_checkpoint_file(file_name=MODEL_PATH, tensor_name='', all_tensors=False)

ae_bottleneck/W (DT_FLOAT) [1,1,128,16]
ae_bottleneck/W/Adam (DT_FLOAT) [1,1,128,16]
ae_bottleneck/W/Adam_1 (DT_FLOAT) [1,1,128,16]
ae_bottleneck/biases (DT_FLOAT) [16]
ae_bottleneck/biases/Adam (DT_FLOAT) [16]
ae_bottleneck/biases/Adam_1 (DT_FLOAT) [16]
ae_dilatedconv_1/W (DT_FLOAT) [1,3,128,128]
ae_dilatedconv_1/W/Adam (DT_FLOAT) [1,3,128,128]
ae_dilatedconv_1/W/Adam_1 (DT_FLOAT) [1,3,128,128]
ae_dilatedconv_1/biases (DT_FLOAT) [128]
ae_dilatedconv_1/biases/Adam (DT_FLOAT) [128]
ae_dilatedconv_1/biases/Adam_1 (DT_FLOAT) [128]
ae_dilatedconv_10/W (DT_FLOAT) [1,3,128,128]
ae_dilatedconv_10/W/Adam (DT_FLOAT) [1,3,128,128]
ae_dilatedconv_10/W/Adam_1 (DT_FLOAT) [1,3,128,128]
ae_dilatedconv_10/biases (DT_FLOAT) [128]
ae_dilatedconv_10/biases/Adam (DT_FLOAT) [128]
ae_dilatedconv_10/biases/Adam_1 (DT_FLOAT) [128]
ae_dilatedconv_11/W (DT_FLOAT) [1,3,128,128]
ae_dilatedconv_11/W/Adam (DT_FLOAT) [1,3,128,128]
ae_dilatedconv_11/W/Adam_1 (DT_FLOAT) [1,3,128,128]
ae_dilatedconv_11/biases (DT_FLOAT

In [38]:
np.intersect1d(varlist, old_varlist)

array(['ae_bottleneck/W', 'ae_bottleneck/biases', 'ae_dilatedconv_1/W',
       'ae_dilatedconv_1/biases', 'ae_dilatedconv_10/W',
       'ae_dilatedconv_10/biases', 'ae_dilatedconv_11/W',
       'ae_dilatedconv_11/biases', 'ae_dilatedconv_12/W',
       'ae_dilatedconv_12/biases', 'ae_dilatedconv_13/W',
       'ae_dilatedconv_13/biases', 'ae_dilatedconv_14/W',
       'ae_dilatedconv_14/biases', 'ae_dilatedconv_15/W',
       'ae_dilatedconv_15/biases', 'ae_dilatedconv_16/W',
       'ae_dilatedconv_16/biases', 'ae_dilatedconv_17/W',
       'ae_dilatedconv_17/biases', 'ae_dilatedconv_18/W',
       'ae_dilatedconv_18/biases', 'ae_dilatedconv_19/W',
       'ae_dilatedconv_19/biases', 'ae_dilatedconv_2/W',
       'ae_dilatedconv_2/biases', 'ae_dilatedconv_20/W',
       'ae_dilatedconv_20/biases', 'ae_dilatedconv_21/W',
       'ae_dilatedconv_21/biases', 'ae_dilatedconv_22/W',
       'ae_dilatedconv_22/biases', 'ae_dilatedconv_23/W',
       'ae_dilatedconv_23/biases', 'ae_dilatedconv_24/W',
   

In [31]:
MODEL_PATH = "/home/julia/DeepVoice_project/LOGDIR/model.ckpt-2"    

reader = pywrap_tensorflow.NewCheckpointReader(MODEL_PATH)

var_to_shape_map  = reader.get_variable_to_shape_map()
for key,value in sorted(var_to_shape_map.items()):
    print("{}: {}".format(key, value))

ae_startconv/W: [1, 3, 1, 128]


In [22]:
ae_layers = [key for key in var_to_shape_map.keys() if key.startswith('ae')]

In [23]:
ae_layers[:3]

['ae_startconv/biases', 'ae_startconv/W/Adam_1', 'ae_startconv/W']

In [26]:
reader.get_tensor(ae_layers[2])

array([[[[ 0.01203278, -0.10520291,  0.12341633, -0.09523151,
           0.09587987, -0.02144042,  0.0783055 , -0.02178621,
          -0.01088013, -0.0614558 , -0.07129394, -0.00432002,
           0.08259791, -0.10954858, -0.11433904, -0.09195298,
          -0.02012742, -0.10045623,  0.09914373,  0.07430148,
           0.04196946,  0.12193274,  0.05003662, -0.08656662,
           0.01942925, -0.11667161, -0.12117441, -0.07204361,
           0.08887656, -0.11284521, -0.02322815, -0.09519912,
           0.10204411,  0.12054746, -0.06709029,  0.02591731,
          -0.00541955, -0.04868354, -0.11747795, -0.1067972 ,
          -0.05294297, -0.0428362 , -0.017465  ,  0.00290056,
          -0.01198498,  0.07556932,  0.10766061,  0.06721687,
          -0.10235035, -0.04098432,  0.00133857, -0.01291195,
           0.01132475,  0.12098466,  0.06837414,  0.0697061 ,
          -0.09864458,  0.08200732,  0.02888057, -0.11723948,
           0.07483652, -0.05749021,  0.03027664, -0.0310448 ,
        

In [14]:
! tensorboard --logdir=/home/julia/DeepVoice_project/LOGDIR/

/home/julia/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/julia/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
TensorBoard 0.4.0rc3 at http://julia-ThinkPad-T470:6006 (Press CTRL+C to quit)
^C


In [64]:
import pandas as pd

def foo(num_l, num_s):
#     print(num_l, num_s)
    res = {}
    for l in range(num_l):
        d = 2**(l%num_s)
#         print('\t {}: {}'.format(l, d))
        res.update({l: d})
    return np.array(res)

# for num_l in range(1, 8):
#     for num_s in range(1, 8):
#         foo(num_l, num_s)
# df = pd.DataFrame(columns = ['num_layers', 'num_stages'])
df = {(num_l, num_s): foo(num_l, num_s) for num_l in range(8, 9) for num_s in range(4, 5)}
df = pd.DataFrame.from_dict(df, orient = 'index')
df[0].values

array([array({0: 1, 1: 2, 2: 4, 3: 8, 4: 1, 5: 2, 6: 4, 7: 8}, dtype=object)],
      dtype=object)

In [34]:
len(df.keys())

49

In [20]:
tf.reset_default_graph()
x = tf.Variable(tf.ones((1, 2)), name = 'slit')
y = tf.Variable(tf.ones((1, 2)), name = 'slit2')
z = tf.Variable(tf.ones((1, 2)), name = 'slit3')
s = tf.Variable(tf.ones((1, 2)), name = 'pslit')
t = tf.Variable(tf.ones((1, 2)), name = 'pslit2')

varlist = tf.global_variables()

In [22]:
[v.op.name for v in varlist if not v.name.startswith('s')]

['pslit', 'pslit2']